In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from IPython.core.debugger import set_trace
%matplotlib notebook

In [14]:
def find_nearest_idx(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [15]:
BASE_PATH = './pack/data/matches/'

class Game():
    def __init__(self, game_id):
        self.game_id = game_id
        self.players = None # Dictionary with keys of player ids
        self.events = None # List of events
        self.id_data = None
        self.id_kills = None
        self.constructGame(game_id)
        
    def constructGame(self, game_id):
        dataFile = BASE_PATH + game_id + '/ticks.csv'
        id_data = pd.read_csv(dataFile)
        self.id_data = id_data
        
        killFile = BASE_PATH + game_id + '/kills.csv'
        id_kills = pd.read_csv(killFile)
        self.id_kills = id_kills
        
        self.loadPlayers()
        self.loadEvents()
    
    def loadPlayers(self):
        killer_ids = np.unique(self.id_kills['killer_recording_id'])

        killers = {}
        for killer in killer_ids:
            kills = self.id_kills['killer_recording_id'] == killer
            killers[killer] = self.id_kills[kills]

        player_ids = np.unique(self.id_data['recording_id'])

        players = {}

        for player_id in player_ids:
            player = Player(player_id)

            # Save player's ticks
            tick_indices = self.id_data['recording_id'] == player_id
            player.tick_indices = tick_indices

            # gathering all the player's kills
            kill_indices = self.id_kills['killer_recording_id'] == player_id
            player.kill_indices = kill_indices

            # Save kill timestamps
            kill_stamps = self.player_kill_ticks(player.tick_indices, player.kill_indices)
            player.kill_stamps = kill_stamps

            players[player_id] = player

        self.players = players
    
    def loadEvents(self):

        def event_tick_index(player_id, event_stamp):
            ticks_indices = self.id_data['recording_id'] == player_id
            player_timestamps = self.id_data['timestamp'][ticks_indices]
            tick_index = find_nearest_idx(player_timestamps, event_stamp)
            return tick_index

        # Construct events
        events = {}
        for idx in range(0, len(self.id_kills)):
            row = self.id_kills.iloc[idx]
            event_id = idx
            killer_id = row['killer_recording_id']
            victim_id = row['victim_recording_id']
            kill_stamp = row['timestamp']
            killer_tick_index = event_tick_index(killer_id, kill_stamp)
            victim_tick_index = event_tick_index(victim_id, kill_stamp)
            event = Event(event_id, killer_tick_index, victim_tick_index, killer_id, victim_id)
            events[event_id] = append(event)
        
        self.events = events
        
    def player_kill_ticks(self, tick_indices, kill_indices):

        kill_stamps = self.id_kills['timestamp'][kill_indices]
        timestamps = self.id_data['timestamp'][tick_indices]
        kill_ticks_stamps = []
        for i, kill_stamp in enumerate(kill_stamps):    
            kill_tick_idx = find_nearest_idx(timestamps, kill_stamp)
            kill_ticks_stamps.append(timestamps[kill_tick_idx])
        return kill_ticks_stamps
    
    def player_log(self, player_id):
        player = self.players[player_id]
        moves = player.list_moves(self.id_data)
        return moves
    
    def build_game_log(self):
        log = {}
        for player_id in players:
            log[player_id] = player_log(player_id)
        return log

In [16]:
class Player():
    def __init__(self, player_id):
        self.player_id = player_id
        self.tick_indices = None
        self.kill_indices = None
        self.kill_stamps = None
    
    def list_moves(self, id_data):
        player_ticks = id_data[self.tick_indices]
        moves = {}
        for step in player_ticks:
            timestep = step['timestep']
            moves[timestep] = {}
            moves[timestep]['x_pos'] = step['x_pos']
            moves[timestep]['y_pos'] = step['y_pos']
        return moves
    
    def list_kills(self, kill_data):
        kills =  

In [17]:
class Event():
    # Kill event
    def __init__(self, event_id, killer_tick_index, victim_tick_index, killer_id, victim_id):
        self.event_id = event_id
        self.killer_id = killer_id
        self.victim_id = victim_id
        self.killer_tick_index = killer_tick_index # Index for tick for killer
        self.victim_tick_index = victim_tick_index
        
    def getKillerTimestamp(self):
        return id_data.iloc[self.killer_tick_index]['timestamp']
    
    def getVictimTimestamp(self):
        return id_data.iloc[self.victim_tick_index]['timestamp']
    
    def player_kill_pos():
        pos_x = id_data['pos_x'][self.killer_tick_index]
        pos_y = id_data['pos_y'][self.killer_tick_index]
        return float(pos_x), float(pos_y)
    
    def player_death_pos():
        pos_x = id_data['pos_x'][self.victim_tick_index]
        pos_y = id_data['pos_y'][self.victim_tick_index]
        return float(pos_x), float(pos_y)
        

In [18]:
id_data = pd.read_csv(
    './pack/data/matches/37549105/ticks.csv')
id_data.head(10)

,Unnamed: 0,recording_id,type_id,timestamp,player_id,pos_x,pos_y,pos_z,rot_x,rot_y,rot_z,flags,speed,vehicle,recording_id.1,session_id,timestamp.1
0,0,3449608,7,1510687274690,3020,-31907.500000,-77257.570312,-2013.723022,1.000000,0.000000,0.000000,0.0,NaN,NaN,3449608,37549105,1510687274690
1,1,3449608,7,1510687275053,3020,-31907.500000,-77257.570312,-2013.723022,0.999998,0.000000,0.002109,0.0,0.000000,NaN,3449608,37549105,1510687274690
2,2,3449608,7,1510687275378,3020,-31907.500000,-77257.570312,-2013.723022,0.495481,-0.860166,-0.120888,0.0,0.000000,NaN,3449608,37549105,1510687274690
3,3,3449608,7,1510687275772,3020,-31907.500000,-77257.570312,-2013.723022,-0.591913,-0.799223,-0.104313,0.0,0.000000,NaN,3449608,37549105,1510687274690
4,4,3449608,7,1510687276141,3020,-31907.500000,-77257.570312,-2013.723022,-0.776267,-0.606164,-0.173134,0.0,0.000000,NaN,3449608,37549105,1510687274690
5,5,3449608,7,1510687276472,3020,-31927.400391,-77262.085938,-2013.723022,-0.958070,-0.177428,-0.224991,0.0,0.061650,NaN,3449608,37549105,1510687274690
6,6,3449608,7,1510687276805,3020,-31945.093750,-77255.718750,-2013.723022,-0.958070,-0.177428,-0.224991,0.0,0.056469,NaN,3449608,37549105,1510687274690
7,7,3449608,7,1510687277181,3020,-31953.791016,-77240.585938,-2013.723022,-0.958070,-0.177428,-0.224991,0.0,0.046420,NaN,3449608,37549105,1510687274690
8,8,3449608,7,1510687277617,3020,-31955.343750,-77239.000000,-2013.723022,-0.959373,-0.180716,-0.216669,0.0,0.005091,NaN,3449608,37549105,1510687274690
9,9,3449608,7,1510687278014,3020,-31945.777344,-77251.078125,-2013.723022,-0.003175,-0.946224,-0.323497,0.0,0.038810,NaN,3449608,37549105,1510687274690


1510687275053

,Unnamed: 0,killer_recording_id,victim_recording_id,timestamp,recording_id,session_id,timestamp.1
0,0,3449687,3443142,1510688133632,3449687,37549105,1510687281295
1,1,3463042,3443144,1510689278868,3463042,37549105,1510688922757
2,2,3448659,3443145,1510687337946,3448659,37549105,1510687147078
3,3,3443257,3443146,1510687547831,3443257,37549105,1510686502645
4,4,3443195,3443177,1510686720106,3443195,37549105,1510686496939
5,5,3443198,3443179,1510686752808,3443198,37549105,1510686497180
6,6,3444818,3443180,1510686832847,3444818,37549105,1510686691851
7,7,3444831,3443181,1510686796092,3444831,37549105,1510686692522
8,8,3451955,3443182,1510687872292,3451955,37549105,1510687583182
9,9,3443186,3443183,1510686925720,3443186,37549105,1510686496939


In [19]:
# Construct game
game = Game('37549105')

In [ ]:
# plot kills

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
# ([0, 0, 1, 1])
bg_figname = 'minimap_true.png'
img = plt.imread(bg_figname)
axis_extent = (-246400, 159990,
               -266400, 140000)
ax.imshow(img, extent=axis_extent)
for player_key in players:
    player = players[player_key]
    pos_x =  []
    pos_y =  []
    #player_kill_stamps = player['kill_stamps']
    player_kill_stamps = player.kill_stamps
    if player_kill_stamps:
        indices = []
        for stamp in player_kill_stamps:
            x,y = player_kill_pos(player_key, stamp)
            pos_x.append(x)
            pos_y.append(y)

        ax.scatter(pos_x, pos_y, c='r', s=5)

In [ ]:
# plot movement
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
# ([0, 0, 1, 1])
bg_figname = 'minimap_true.png'
img = plt.imread(bg_figname)
axis_extent = (-246400, 159990,
               -266400, 140000)
ax.imshow(img, extent=axis_extent)

for player_key in players:
    player = players[player_key]
    player_indices = player.tick_indices
    pos_x = id_data['pos_x'][player_indices]
    pos_y = id_data['pos_y'][player_indices]
    ax.scatter(pos_x, pos_y, c=np.random.rand(3,), s=1)

In [ ]:
# Combined plot


In [ ]:
np.max(id_data['pos_y'])

pos_x = id_data['pos_x']
pos_y = id_data['pos_y']

# player ids

players = {}
for player in players:
    print(player)
    player_dict = {}
    tick_indices = id_data['recording_id'] == player
    player_ticks = id_data[tick_indices]
    
    try:
        player_kills = killers[player]
    except:
        # No kills
        continue
    player_tick_stamps = np.array(player_ticks['timestamp'].values.tolist())
    kill_indices = []
    kill_stamps = player_kills['timestamp']
    for kill_stamp in kill_stamps.values():
        # kill_tick_idx = find_nearest_idx(player_tick_stamps, kill_stamp)
        # corresponding_player_tick = player_ticks.iloc[kill_tick_idx]
        kill_tick_idx = id_data['timestamp'] == kill_stamp
        kill_indices.append(kill_tick_idx)
    
    player_dict['tick_indices'] = tick_indices
    player_dict['kill_indices'] = kill_indices
    
    
    players[player] = player_dict